In [1]:
import pandas as pd
import os
import ast
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import re 
from networkx.algorithms.community.quality import coverage, performance
import networkx.algorithms.community as nx_comm
from networkx.algorithms.centrality import edge_betweenness_centrality as edge_bet
from networkx.algorithms.shortest_paths.generic import average_shortest_path_length as len_short_path
import numpy as np
import pprint
from typing import Dict
import json
from tqdm.notebook import tqdm
import community
# import nxmetis
import time
from math import ceil
from sklearn import metrics
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer
import os
import nltk
from afinn import Afinn
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from collections import Counter
import itertools
from nltk.corpus import stopwords
import string
from nltk import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer
import networkx as nx
import re 
import pickle
from sklearn.preprocessing import MinMaxScaler
#import igraph as ig
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim
from nltk.stem import PorterStemmer 
from networkx.algorithms.centrality import degree_centrality
import random
np.random.seed(2018)
from gensim.models import CoherenceModel
# https://towardsdatascience.com/graph-algorithms-part-2-dce0b2734a1d
# https://github.com/taynaud/python-louvain/blob/master/community/community_louvain.py
# https://networkx.github.io/documentation/stable/_modules/networkx/algorithms/community/quality.html
# https://stackoverflow.com/questions/929103/convert-a-number-range-to-another-range-maintaining-ratio
from langdetect import detect

### Per aggiungere eventualmente altri dati

In [2]:
os.chdir('./data/corona_virus')

In [ ]:
multi = nx.read_gml('Graph/Final_MultiGraph_Covid.gml')

In [ ]:
print(nx.info(multi))

In [ ]:
graph = nx.read_gml('Graph/Final_Graph_Covid.gml')

In [ ]:
print(nx.info(graph))

In [ ]:
CG = nx.read_gml('Graph/Final_DiGraph_Covid.gml')

In [ ]:
print(nx.info(CG))

In [3]:
new_data = pd.read_csv('./Dataset_marzo_JSON/slot3.csv')
new_data.dropna(axis='index', how='all', subset=['text'], inplace = True)

In [4]:
new_data.shape

(2114411, 12)

In [7]:
language = list()
cont = 0
for _, row in tqdm(new_data.iterrows()):
    try:
        language.append(detect(row['text']))
    except:
        language.append('unknown')

In [8]:
new_data['lang'] = language

In [9]:
final_eng_march = new_data.query("lang == 'en'")

In [17]:
final_eng_march.shape

(1178531, 14)

In [11]:
final_eng_march = final_eng_march.rename(columns={"lang": "language", "text": "text_con_hashtag" })

In [12]:
def find_mentions(x):
    p = re.compile(r'@\s([^\s:]+)')
    return  ', '.join(p.findall(x))

def remove_mentions(x):
    p = re.sub(r'@\s([^\s:]+)', '', x)
    return p

final_eng_march["@mentions"] = final_eng_march["text_con_hashtag"].apply(find_mentions)
final_eng_march["text_con_hashtag"] = final_eng_march["text_con_hashtag"].apply(remove_mentions)
final_eng_march["@mentions"] = final_eng_march["@mentions"].str.lower()
final_eng_march["username"] = final_eng_march["username"].str.lower()

In [16]:
final_eng_march = final_eng_march.drop_duplicates(subset=['text_con_hashtag', '@mentions', 'username', 'retweets', 'favorites'], keep='last')

In [18]:
final_eng_march = final_eng_march[['username', 'favorites', 'retweets', 
        'language', '@mentions', 'geo', 'text_con_hashtag']]

In [19]:
final_eng_march.head(5)

,username,favorites,retweets,language,@mentions,geo,text_con_hashtag
5,hccons_,4,2,en,,Republic of Texas,"It’s Coronavirus day 1,387. All the government..."
6,sarahplz,71,11,en,,sc: sarahplz10,If you ever lived in woodcrest you are immune ...
8,thomnolan,1,0,en,,Roscommon,As far as I can tell not one corona virus case...
11,rexcorvinus,0,0,en,,Freedomland,"Even w/o the coronavirus, ""longest bull market..."
12,bondjam78417173,0,0,en,,false,Trump's Top Coronavirus Doctor Says It's Much ...


In [20]:
final_eng_march.to_csv('./last_data.csv')